In [ ]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import tensorflow as tf

dataset_labels = pd.read_csv("../input/dog-breed-identification/labels.csv")

In [ ]:
from IPython.display import Image 
Image("../input/dog-breed-identification/train/000bec180eb18c7604dcecc8fe0dba07.jpg")

In [ ]:
dataset_labels.head()

In [ ]:
filenames =["../input/dog-breed-identification/train/"+fname+".jpg" for fname in dataset_labels["id"]]
filenames[:5]

In [ ]:
##checking for likeliness
import os
if len(os.listdir("../input/dog-breed-identification/train/")) == len(filenames):
    print("okay")

In [ ]:
labels =dataset_labels["breed"]

In [ ]:
labels = dataset_labels["breed"].to_numpy()
labels

In [ ]:
##checking for likeness 
if len(labels) == len(filenames):
    print("okay")

In [ ]:
unique_labels =np.unique(labels)
len(unique_labels)

In [ ]:
 ##Turning labels into boolean arrays
boolean_labels = [label == unique_labels for label in labels]

In [ ]:
 ## Turning boolean arrays into integer
boolean_labels[2].astype(int)

### creating validation set from the train set

In [ ]:
x = filenames
y = boolean_labels
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x[:1000],
                                               y[:1000],
                                              test_size=0.2,
                                              random_state= 42)
len(x_train),len(x_val),len(y_train),len(x_val)

## Turning images  and labels into tensors

To process image we need
*  Take image filapath as input
*  Use tenseflow to read the file and save it to a variable , 'image'(the variable will be a tensor of type string)
*  Turn 'image' (jpg) into  numerical tensor
*  convert the colour channel values from 0-255  to 0-1(scalling)
*  Resize the 'image' to be a shape of (224,224)
*  Return the modified image

In [ ]:
IMG_size = 24
def process_image(image_path):
        ##reading image path
        image =tf.io.read_file(image_path)
        ##turn image into tensor
        image =tf.image.decode_jpg(image,channels=3)
        ##normalization of image
        image =tf.image.convert_image_dtype(image,tf.float32)
        ##resize the image
        image =tf.image.resize(image,size=[IMG_size,IMG_size])
        return image 

In [ ]:

def get_image_label(image_path,label):
    image=process_image(image_path)
    return image,label
    

## Turning image set into batch

In [ ]:
BATCH_SIZE = 32
def data_batches(x,y=None,batch_size=BATCH_SIZE,valid_data=False,test_data=False):
    if test_data:
        print("creating a test batch...")
        data =tf.data.dataset.from_tensor_slices((tf.constant(x)))
        data_batch=data.map(process_image).batch(batch_size)
        return data_batch
    elif valid_data:
        print("creating valid data batch...")
        data=tf.data.dataset.from_tensor_slices((tf.constant(x)),
                                                (tf.constant(y)))
        data_batch=data.map(get_image_label).batch(batch_size)
        return data_batch
    else:
        data = tf.data.dataset.from_tensor_slices((tf.constant(X)),
                                                  (tf.constant(y)))
        data = data.shuffle(buffer_size=len(x))
        data_batches=data.map(get_image_label).batch(batch_size)
    return data_batch    